In [45]:
### full data analysis and string assembly for HTML implementation ###

''' Point and Kill-based minigames, same regex pattern applies 
    1. Walls
    2. Mini SG 
    3. Mini SW
    4. Lasertag
    5. Mienengefecht
    6. OITC
    7. Paintball
    8. Spleef 
    9. Reihenfolge 
    10. Schießstand
    11. Sammelwahn
    12. Buntes Chaos
    13. Duelle 
'''

import os 
import re
import pandas as pd 
import pyperclip as pc

directory = "in/"
fileList = [os.path.join(directory, file) for file in os.listdir(directory)]

def getPlayerName(path):  
    name_pattern = re.compile(r"/([a-zA-Z0-9_]+)\.txt")
    match_name = name_pattern.search(path)

    # Extracted name
    extracted_name = match_name.group(1) if match_name else None
    
    return extracted_name

playerList = []

for file in fileList:
    player = getPlayerName(file)
    playerList.append(player)


def getPointRecord(filepath, minigame):
    records = []
    
    if type(filepath) == str:
        try: 
            fh = open(filepath, 'r')
        except: 
            print("Invalid Filepath")
            return None

        for line in fh:
            if minigame in line:
                player_name = getPlayerName(filepath)

                pattern = r"literal\{([^}]*)\}\[style=\{\}\], literal\{: \}\[style=\{\}\]\].+?literal\{(\d+)\}\[style=\{\}\]"
                matches = re.findall(pattern, line)         


                #for map_name, value in matches:
                    #if value != None:
                        #try: value = int(matches)
                        # except: print(value, "could not be converted to an integer")

                # Creating a dictionary from the matches
                data = {map_name: int(value) for map_name, value in matches}

                # Converting the dictionary into a DataFrame
                df = pd.DataFrame([data], index=[player_name])

                return df

        fh.close()


def getHtmlPointRecords(minigame):
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getPointRecord(filePath, minigame)
            combined_df = pd.concat([combined_df, temp_df], axis=0)
        
    return combined_df.fillna(0).   to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '').replace(".0<", "<")

def getDataFramePointRecords(minigame):
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getPointRecord(filePath, minigame)
            combined_df = pd.concat([combined_df, temp_df], axis=0)
            combined_df.fillna(0, inplace=True)

    return combined_df.fillna(0)   

'''
    Functions to enable compatibility with Time-based minigames (aside from JnR)
'''

def parseTime(minutes, seconds, milliseconds):
    # If minutes is None or empty, default to '0'
    if not minutes:
        minutes = '0'
    return f"{minutes}:{seconds}\"{milliseconds}"

def getMapRecord(filepath, minigame):
    if type(filepath) == str:
        try: 
            with open(filepath, 'r') as fh:
                file_content = fh.readlines()
        except: 
            print("Invalid Filepath")
            return None

        for line in file_content:
            if minigame in line:
                # Regex pattern for times greater than 1 minute
                pattern_gt_1min = r"literal\{([^}]*)\}\[style=\{\}\], literal\{: \}\[style=\{\}\].+?literal\{(\d+)\}\[style=\{\}\], literal\{min \}\[style=\{\}\], literal\{(\d+)\}\[style=\{\}\], literal\{s \}\[style=\{\}\], literal\{(\d+)\}\[style=\{\}\]"
                # Regex pattern for times less than 1 minute
                pattern_lt_1min = r"literal\{([^}]*)\}\[style=\{\}\], literal\{: \}\[style=\{\}\].+?literal\{(\d+)\}\[style=\{\}\], literal\{s \}\[style=\{\}\], literal\{(\d+)\}\[style=\{\}\]"
                
                # Try matching pattern for times greater than 1 minute
                matches = re.findall(pattern_gt_1min, line)
                if not matches:
                    # Try matching pattern for times less than 1 minute
                    matches = re.findall(pattern_lt_1min, line)
                    # Adjust match structure to fit parseTime function
                    matches = [(m[0], None, m[1], m[2]) for m in matches]

                if matches:
                    player_name = getPlayerName(filepath)
                    data = {map_name.strip(): parseTime(*time_parts) for map_name, *time_parts in matches}
                    df = pd.DataFrame([data], index=[player_name])
                    
                    if "rache" in minigame: return df.fillna('0:00"000')
                    return df
    return None

def GetJnRRecords(minigame, html=False):
    combined_df = getDataFrameTimeRecords(minigame) 

    canyon_df = combined_df[['Canyon']].copy() if 'Canyon' in combined_df.columns else pd.DataFrame()

    # modifing data in place to remove canyon after initial df has been created (canyon >1 min times)
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-REPLACEMENT')
            file_data = file_data.replace('[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{2}[style={}], literal{min }[style={}], literal{', 'TEMP-REPLACEMENT')

            with open(filePath, 'w') as file:
                file.write(file_data)

    # getting other map times 
    modified_df = getDataFrameTimeRecords(minigame)

    final_df = pd.concat([canyon_df, modified_df], axis=1)

    # restoring data
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('TEMP-REPLACEMENT', '[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-REPLACEMENT', '[literal{Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{2}[style={}], literal{min }[style={}], literal{')

            with open(filePath, 'w') as file:
                file.write(file_data)

    if html==False: return final_df
    if html==True: return final_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')

def GetWettrRecords(minigame='Wettr', html=False):
    combined_df = getDataFrameTimeRecords(minigame) 

    nonSnow_df = combined_df[['Green Hills', 'Cherry Blossom Canyon', 'Cyberpunk']].copy() if 'Green Hills' in combined_df.columns else pd.DataFrame()

    # modifing data in place to remove canyon after initial df has been created (canyon >1 min times)
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('siblings=[literal{Green Hills}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-GH-REPLACEMENT')
            file_data = file_data.replace('siblings=[literal{Cherry Blossom Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-CB-REPLACEMENT')
            file_data = file_data.replace('siblings=[literal{Cyberpunk}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-CP-REPLACEMENT')
            file_data = file_data.replace('siblings=[literal{Islands}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{', 'TEMP-IS-REPLACEMENT')

            with open(filePath, 'w') as file:
                file.write(file_data)

    # getting other map times 
    modified_df = getDataFrameTimeRecords(minigame)

    final_df = pd.concat([nonSnow_df, modified_df], axis=1)

    # restoring data
    for filePath in fileList:
        if "txt" in filePath:
            with open(filePath, 'r') as file:
                file_data = file.read()

            file_data = file_data.replace('TEMP-GH-REPLACEMENT', 'siblings=[literal{Green Hills}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-CB-REPLACEMENT', 'siblings=[literal{Cherry Blossom Canyon}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-CP-REPLACEMENT', 'siblings=[literal{Cyberpunk}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')
            file_data = file_data.replace('TEMP-IS-REPLACEMENT', 'siblings=[literal{Islands}[style={}], literal{: }[style={}]]], empty[style={color=gold}, siblings=[literal{1}[style={}], literal{min }[style={}], literal{')

            with open(filePath, 'w') as file:
                file.write(file_data)

    if html==False: return final_df
    if html==True: return final_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')


def getDataFrameTimeRecords(minigame):   
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getMapRecord(filePath, minigame)
            if temp_df is not None:
                combined_df = pd.concat([combined_df, temp_df], axis=0)

    return combined_df

def getHtmlTimeRecords(minigame):   
    combined_df = pd.DataFrame()

    for filePath in fileList:
        if "txt" in filePath:
            temp_df = getMapRecord(filePath, minigame)
            if temp_df is not None:
                combined_df = pd.concat([combined_df, temp_df], axis=0)
    
    if minigame in "Drachenflucht": return combined_df.fillna('0:00"000').to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')
    if minigame in "Replika": return combined_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '').replace('0:8"','0:08"').replace('0:7"','0:07"').replace('0:9"','0:09"').replace('0:6"','0:06"').replace('0:5"','0:05"').replace('0:4"','0:04"').replace('0:3"','0:03"').replace('0:2"','0:02"')
    else: return combined_df.to_html().replace('\n', '').replace('<table border="1" class="dataframe">', '').replace('</table>', '')

def getPointRecordRank(minigame, abs=False):
    if abs == True: return getDataFramePointRecords(minigame).rank(ascending=False, na_option='bottom', method='dense').sum(axis=1).sort_values()
    else: return getDataFramePointRecords(minigame).rank(ascending=False, na_option='bottom', method='dense').sum(axis=1).rank(method='min').sort_values()

miniGameList = ["1-Sammelwahn",
        "2-Schießstand",
        "3-Wettrennen",
        "4-Pferderennen",
        "5-Drachenflucht",
        "6-Jump and Run",
        "7-Hoch Hinaus",
        "8-Blockhüpfer",
        "9-Todeswürfel",
        "10-Freier Fall",
        "11-Elytrarennen",
        "12-Waffenfolge",
        "13-Minenfeld",
        "14-Kletterkönig",
        "15-Ampelrennen",
        "16-Replika",
        "17-Walls",
        "18-Mini SG",
        "19-Mini SW",
        "20-Lasertag",
        "21-Minengefecht",
        "22-OITC",
        "23-Paintball",
        "24-Spleef",
        "25-Buntes Chaos",
        "26-Reihenfolge",
        "27-Duelle"]

queryList = ["Sammel",
        "Schie",
        "Wettr",
        "Pferde",
        "Drache",
        "Jump",
        "Hoch",
        "Blockh",
        "Todesw",
        "Freie",
        "Elytra",
        "Waffe",
        "Minenf",
        "Kletter",
        "Ampelrennen",
        "Repli",
        "Walls",
        "Survival",
        "Mini Skyw",
        "Lasertag",
        "Mineng",
        " im ",
        "Paintball",
        "Spleef",
        "Chaos",
        "Reihe",
        "Duel"]

pointMiniGames = "Sammelwahn, Schießstand, Walls, Mini Survival Games, Mini Skywars, Lasertag, Minengefecht, Einer im Köcher, Paintball, Spleef, Buntes Chaos, Reihenfolge, Duelle" # contains Kill Minigames asw as they use the same functions 
timeMiniGames = "Pferderennen, Drachenflucht, Replika, Hoch Hinaus, Blockhüpfer, Todeswürfel, Freier Fall, Elytrarennen, Waffenfolge, Minenfeld, Kletterkönig, Ampelrennen" # the rest, but JnR and Wettrennen


frontWrapperList = []
x = 0 

for i in miniGameList: # assembling heads of table wrappers for readability and compatibility with the rest of the code. 
    import pyperclip as pc
    x = x + 1
    strX = str(x)
    frontWrapperList.append('<!-- Table: ' + i + ' --> <div id="table' + strX + '" style="display:none;"> <table id="dataframe' + strX + '" class="display nowrap">')

endWrapper = "</table> </div>"

def assembleHtmlString(query, i): # assemble HTML codes for desired Minigames
    if query in pointMiniGames:
        print(frontWrapperList[i])
        # print("query", query)             # debug
        print(getHtmlPointRecords(query))
        print(endWrapper)
    if query in timeMiniGames: 
        print(frontWrapperList[i])
        # print("query", query)             # debug
        print(getHtmlTimeRecords(query))
        print(endWrapper)
    if query in "Jump and Run":
        print(frontWrapperList[i])
        print(GetJnRRecords(query, html=True))
        print(endWrapper)
    if query in "Wettrennen":
        print(frontWrapperList[i])
        print(GetWettrRecords(html=True))
        print(endWrapper)
    if query not in timeMiniGames:
        if query not in pointMiniGames: 
            if query not in "Jump and Run":
                if query not in "Wettrennen":
                    raise ValueError("Query not in Supported Minigames (check language?)")

x = 0
outputString = ""

print('<div class="table-container">')
for i in queryList: 
    assembleHtmlString(query=i, i=x) 
    x = x + 1 
print('</div>')


<div class="table-container">
<!-- Table: 1-Sammelwahn --> <div id="table1" style="display:none;"> <table id="dataframe1" class="display nowrap">
  <thead>    <tr style="text-align: right;">      <th></th>      <th>Kiefern-Urtaiga</th>      <th>Birken-Urwald</th>      <th>Verschneite Taiga</th>      <th>Sumpf</th>      <th>Taiga</th>      <th>Fichten-Urtaiga</th>      <th>Ebene</th>      <th>Bambusdschungel</th>      <th>Kirschberghain</th>      <th>Birkenwald</th>      <th>Lichter Dschungel</th>      <th>Wald</th>      <th>Dschungel</th>      <th>Zerzauste Savanne</th>      <th>Zerzauste HÃ¼gel</th>      <th>Verschneite Ebene</th>      <th>Savannenhochebene</th>      <th>Savanne</th>      <th>Dichter Wald</th>      <th>Blumenwald</th>      <th>Sonnenblumenebene</th>      <th>Karmesinwald</th>      <th>Wirrwald</th>      <th>Berghain</th>      <th>Zerzauste Hügel</th>      <th>Alm</th>      <th>Ãœppige HÃ¶hlen</th>      <th>Abgetragene Tafelberge</th>      <th>Tropfsteinhöhlen</th>    

In [46]:
GetWettrRecords(html=False)

,Green Hills,Cherry Blossom Canyon,Cyberpunk,Snow
Anni808,"1:08""768","1:22""912","1:23""467","0:46""496"
Axollotel,"1:12""555",NaN,NaN,"0:48""439"
Bartschii,"1:15""910",NaN,NaN,"0:53""246"
Brilux,"1:08""585","1:29""847","1:21""730","0:46""250"
cediiiiii_10,"1:04""439","1:18""876","1:18""799","0:44""679"
Chander24,"1:12""305","1:27""484","1:26""495","0:52""084"
CrocoCun2020,"1:12""233",NaN,"1:35""768","0:56""393"
crocodile1870,"1:10""543","1:27""831","1:25""019","0:47""878"
cxl1lo,"1:08""494","1:23""103","1:23""377","0:46""667"
DieserBear,"1:07""794","1:23""627","1:23""866","0:45""171"


In [49]:
getPointRecordRank("Sammelwahn", abs=True).head(10)

cediiiiii_10     97.0
Gobo9           118.0
Gerrygames      129.0
HerrDante       130.0
DieserBear      132.0
Ex4cted         140.0
lizsyy          152.0
KakaMC          158.0
PedroZockt      160.0
Grapfen         161.0
dtype: float64

In [50]:
getPointRecordRank("Lasert", abs=True).head(10)

HerrDante        4.0
Fflopse          7.0
Kiritoonroad     8.0
PedroZockt      14.0
cediiiiii_10    18.0
Gobo9           18.0
DieserBear      18.0
Ex4cted         19.0
Grapfen         21.0
_n3d            26.0
dtype: float64

In [51]:
getPointRecordRank("Einer", abs=True).head(10)

Fflopse          4.0
HerrDante        5.0
cediiiiii_10    12.0
DieserBear      13.0
Kiritoonroad    14.0
Gobo9           14.0
Grapfen         17.0
Brilux          22.0
tsuneee         25.0
Ex4cted         28.0
dtype: float64